In [1]:
import os, json, av, PIL
from PIL import Image, ImageOps
from os import environ

videos_location=environ.get("videos_location",None)
group_name=environ.get("group_name",None)
tensorbox_input_filepath=environ.get("tensorbox_input_filepath","../ARThermal/tensorbox_dataset/tensorbox_input.json")
tensorbox_dataset_dir=environ.get("tensorbox_dataset_dir","../ARThermal/tensorbox_dataset")

#pedestrian_log_filepath=environ.get("pedestrian_log_filepath","log_Victor_006_pedestrian.log")
#detector_type=environ.get("detector_type","icf")
pedestrian_log_filepath=environ.get("pedestrian_log_filepath",None)
detector_type=environ.get("detector_type","yolo")
images_to_avoid_filepath=environ.get("images_to_avoid_filepath","../ARThermal/tensorbox_dataset/images_to_avoid.txt")

In [2]:
'''return a dictionary with rgb_frame_path as keys and an array of boxes as values
   format: {frame_path1:[(x1,y1,width1,height1), ... ], ... }'''
def icf_get_frames_with_boxes(log_filepath):
    with open(log_filepath, "r") as f:
        lines = f.readlines()

    frame_boxes = {}
    for i,line in enumerate(lines):
        if line.startswith("total : "):
            number_of_people = int(line.split()[2])

            if number_of_people == 0:
                continue

            #parse (x, y, width, height) of each box
            boxes = []
            for j in range(1,number_of_people+1):
                #parse (x, y, width, height)
                bounds = [int(item) for item in lines[i-j].split()[:-1]]
                boxes.append(bounds)

            #frame_path = /media/Gui2/thermix/ARThermal/Victor_006_frames_no_movement/15_tim/3/Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_1/0.png 
            frame_path = lines[i+2][:-1]
            
            frame_boxes[frame_path] = boxes

    return frame_boxes

In [3]:
'''return a dictionary with rgb_frame_path as keys and anarray of boxes as values
   format: {frame_path1:[(x1,y1,width1,height1),...], ... }'''
def yolo_get_frames_with_boxes(log_filepath):
    with open(log_filepath, "r") as f:
        lines = f.readlines()

    frame_boxes = {}
    for i,line in enumerate(lines):        
        #line = "filepath ['person', 231.98944, 234.04822, 145.44432, 263.03268, 0.60258942842483521]"
        
        filename = line.split()[0]
        
        #person
        semantic = line.split()[1][2:-2]
        if semantic != "person":
            print semantic
            continue
        
        #bounds = [231.98944, 234.04822, 145.44432, 263.03268]
        bounds = [float(item[:-1]) for item in line.split()[2:-1]]
        score = float(line.split()[-1][:-1])
        
        #if score < 0.5:
        #    print 'low precision at: '+filename
        #    continue
            
        w = bounds[2]//2
        h = bounds[3]//2
        bounds[0] = bounds[0] - w
        bounds[1] = bounds[1] - h
        if frame_boxes.get(filename):
            frame_boxes[filename].append(bounds)
        else:
            frame_boxes[filename] = [bounds]

    return frame_boxes

In [4]:
"""return the thermal frame path that correspond with the given rgb frame path
   return None if thermal frame does not exist on disk"""
def get_thermal_frame_from_rgb(rgb_frame_path):
    #rgb_frame_path= /media/Gui2/thermix/ARThermal/Victor_006_frames_no_movement/15_tim/3/Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_1/0.png
    
    #frame_name = 0.png
    rgb_frame_name = os.path.basename(rgb_frame_path)
    
    rgb_frame_id = int(os.path.splitext(rgb_frame_name)[0])
    
    #rgb_video_path = /media/Gui2/thermix/ARThermal/Victor_006_frames_no_movement/15_tim/3/Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_1
    rgb_video_path = os.path.dirname(rgb_frame_path)
    
    #rgb_video_name = Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_1
    rgb_video_name = os.path.basename(rgb_video_path)
    
    #thermal_video_name = Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_2
    thermal_video_name = rgb_video_name[:-1]+"2"
    
    #category_video = 3
    category_video = os.path.basename(os.path.dirname(rgb_video_path))
    
    #root_dataset_dir = /media/Gui2/thermix/ARThermal/Victor_006_frames_no_movement
    root_dataset_dir = os.path.abspath(os.path.join(rgb_video_path,"../../.."))
    
    #thermal_video_path = /media/Gui2/thermix/ARThermal/Victor_006_frames_no_movement/14_tim/3/Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_2
    thermal_video_path = os.path.join(root_dataset_dir,"14_tim", category_video, thermal_video_name)
    
    #get thermal_frame_id from rgb_frame_id
    rgb_frames_count=len(os.listdir(rgb_video_path))
    thermal_frames_count=len(os.listdir(thermal_video_path))
    
    if thermal_frames_count == 0:
        return None
    
    relation_rgb_thermal = float(rgb_frames_count)/thermal_frames_count
    
    thermal_frame_id = int(round(rgb_frame_id / relation_rgb_thermal))
    
    #thermal_frame_name for corresponding id
    thermal_frame_name = "%d_%d.png" % (int(thermal_frame_id/3), thermal_frame_id%3)
    
    thermal_frame_path = os.path.join(thermal_video_path, thermal_frame_name)
    
    if not os.path.exists(thermal_frame_path):
        return None
    
    return thermal_frame_path

In [5]:
"""Transposing box from rgb detection to thermal frame"""
def get_transpoled_box(box_rgb, shape_rgb, shape_thermal):
    box_thermal = {'x':0, 'y':0, 'w':0, 'h':0}
    
    x1 = box_rgb['x']
    y1 = box_rgb['y']
    w1 = box_rgb['w']
    h1 = box_rgb['h']
    
    w_color = shape_rgb[0]
    h_color = shape_rgb[1]
    
    w_thermal = shape_thermal[0]
    h_thermal = shape_thermal[1]
    
    box_thermal['x'] = w_thermal * x1 / w_color
    box_thermal['y'] = h_thermal * y1 / h_color
    box_thermal['w'] = w_thermal * w1 / w_color
    box_thermal['h'] = h_thermal * h1 / h_color
    
    return box_thermal

In [6]:
"""return a PIL Image with a thermal frame if exists. otherwise return None"""
def get_thermal_frame_of_video(video_path, frame_id):
    try:
        container = av.open(video_path)
        video = next(s for s in container.streams if s.type == b'video')

        #super_frame_id is the frame that contains in one channel the frame_id
        super_frame_id = frame_id//3
        for packet in container.demux(video):
            for frame in packet.decode():
                if frame.index != super_frame_id:
                    continue

                img = frame.to_image()
                img = img.resize((640, 480))

                rgb = img.split()

                channel = rgb[frame_id%3]
                img2 = PIL.Image.merge("RGB", (channel, channel, channel))
                img2 = ImageOps.mirror(img2)
                return img2
    except Exception as e:
        print "Error getting frames of video %s" % video_path
        return None

In [7]:
def create_thermal_frame_from_rgb(rgb_frame_path, videos_dir, group_name, output_dir):
    #rgb_frame_path= /media/Gui2/thermix/ARThermal/Victor_006_frames_no_movement/15_tim/3/Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_1/0.png
    
    #frame_name = 0.png
    rgb_frame_name = os.path.basename(rgb_frame_path)
    
    rgb_frame_id = int(os.path.splitext(rgb_frame_name)[0])
    
    #rgb_video_path = /media/Gui2/thermix/ARThermal/Victor_006_frames_no_movement/15_tim/3/Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_1
    rgb_video_path = os.path.dirname(rgb_frame_path)
    
    #rgb_video_name = Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_1
    rgb_video_name = os.path.basename(rgb_video_path)
    
    #thermal_video_name = Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_2
    thermal_video_name = rgb_video_name[:-1]+"2"
    
    #category_video = 3
    category_video = os.path.basename(os.path.dirname(rgb_video_path))
    
    #root_dataset_dir = /media/Gui2/thermix/ARThermal/Victor_006_frames_no_movement
    root_dataset_dir = os.path.abspath(os.path.join(rgb_video_path,"../../.."))
    
    #thermal_video_path = /media/Gui2/thermix/ARThermal/Victor_006_frames_no_movement/14_tim/3/Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_2
    thermal_video_path = os.path.join(root_dataset_dir,"14_tim", category_video, thermal_video_name)
    
    #get thermal_frame_id from rgb_frame_id
    rgb_frames_count=len(os.listdir(rgb_video_path))
    thermal_frames_count=len(os.listdir(thermal_video_path))
    
    if thermal_frames_count == 0:
        return None
    
    relation_rgb_thermal = float(rgb_frames_count)/thermal_frames_count
    
    thermal_frame_id = int(round(rgb_frame_id / relation_rgb_thermal))
    
    original_thermal_video_path = os.path.join(videos_dir,group_name,"14_tim",thermal_video_name+".mov")
    
    thermal_frame_img = get_thermal_frame_of_video(original_thermal_video_path,thermal_frame_id)
    
    if thermal_frame_img:
        #thermal_frame_name for corresponding id
        thermal_frame_name = "%d_%d.png" % (int(thermal_frame_id/3), thermal_frame_id%3)
        thermal_frame_path = os.path.join(output_dir, thermal_video_name, thermal_frame_name)
        if not os.path.exists(os.path.dirname(thermal_frame_path)):
            os.makedirs(os.path.dirname(thermal_frame_path))
        
        thermal_frame_img.save(thermal_frame_path)
        
        return thermal_frame_path
    else:
        return None

In [8]:
"""
take {'x':181, 'y':138, 'w':16, 'h':18}
return {
        "x1": 181, 
        "x2": 197, 
        "y1": 138, 
        "y2": 156
       }
"""
def rect_to_standard_format(box):
    rect = {}
    rect["x1"] = box["x"]
    rect["x2"] = box["x"]+box["w"]
    rect["y1"] = box["y"]
    rect["y2"] = box["y"]+box["h"]
    
    return rect 

In [9]:
"""
format required by tensorbox:

[
{
"image_path": "brainwash_11_13_2014_images/00005500_640x480.png", 
"rects": [
          {
           "x1": 181.0, 
           "x2": 197.0, 
           "y1": 138.0, 
           "y2": 156.0
          },
         ]
},
]
"""
tensorbox_input = []

images_to_avoid = []
if os.path.exists(images_to_avoid_filepath):
    with open(images_to_avoid_filepath, 'r') as infile:
        images_to_avoid = infile.readlines()
        
#read current tensorbox input
if os.path.exists(tensorbox_input_filepath):
    with open(tensorbox_input_filepath, 'r') as infile:
        tensorbox_input = json.load(infile)

#parse output of pedestrian detector
if detector_type == "icf":
    frames_boxes = icf_get_frames_with_boxes(pedestrian_log_filepath)
elif detector_type == "yolo":
    frames_boxes = yolo_get_frames_with_boxes(pedestrian_log_filepath)
else:
    raise "detector does not exist"

#create tensorbox dataset directory
if not os.path.exists(tensorbox_dataset_dir):
    os.makedirs(tensorbox_dataset_dir)
    print "dataset dir created"
        
for rgb_frame_path in frames_boxes.keys():
    rgb_image = Image.open(rgb_frame_path)
    shape_rgb = rgb_image.size
        
    shape_thermal = (640,480)

    bx = []
    for b in frames_boxes[rgb_frame_path]:
        bx.append({'x':b[0], 'y':b[1], 'w':b[2], 'h':b[3]})

    thermal_boxes = []
    for instance in bx:
        thermal_box = get_transpoled_box(instance, shape_rgb, shape_thermal)
        thermal_boxes.append(thermal_box)

    # creste thermal frame with size (640,480) and saved it in dataset tensorbox directory
    thermal_frame_path = create_thermal_frame_from_rgb(rgb_frame_path, videos_location, group_name, tensorbox_dataset_dir)
    if not thermal_frame_path:
        continue
    
    #frame_if_exists = frame dict if exists in tensorbox_input, otherwise = None
    frame_if_exists = next((item for item in tensorbox_input if item["image_path"] == thermal_frame_path), None)    
    
    #video_and_frame_name = Users_ipod5_unkown_2016-08-03_00%3A01%3A09.000000_2/0_0.png
    video_and_frame_name = os.path.join(os.path.basename(os.path.dirname(thermal_frame_path)), 
                                        os.path.basename(thermal_frame_path))
    if video_and_frame_name in images_to_avoid:
        # if frame exists in tensorbox_input, remove it
        if frame_if_exists:
            tensorbox_input.remove(frame_if_exists)
        continue
        
    # if image result already in tensorbox_input, avoid adding twice
    if frame_if_exists:
        continue
        
    frame_info = {"image_path":thermal_frame_path}
    frame_info["rects"] = []    
    for box in thermal_boxes: 
        frame_info["rects"].append(rect_to_standard_format(box))
        
    tensorbox_input.append(frame_info)

IOError: [Errno 2] No such file or directory: ''

In [10]:
with open(tensorbox_input_filepath, 'w') as outfile:
    json.dump(tensorbox_input, outfile, indent=2)